# Downloading SAMOS data from MarineFlux ERDDAP server

In [1]:
%pip install erddapy
%pip install cf_xarray
%pip install netCDF4
%pip install cartopy

from erddapy import ERDDAP
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import numpy.ma as ma
import cf_xarray

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Create the ERDDAP request

In [2]:
flux_algorithms = ["S88", "B23", "C36"]
extent = [280.0, 300.0, 30.0, 45.0] # [lon-, lon+, lat-, lat+]
start_time = "2015-11-01T00:00:00Z"
end_time = "2015-12-01T00:00:00Z"

In [3]:
erddap_samos = dict()
for algo in flux_algorithms:
  erddap_samos[algo] = ERDDAP(
    server="http://marineflux-erddap.coaps.fsu.edu/erddap",
    protocol="tabledap",
  )

  erddap_samos[algo].dataset_id = f"SAMOS_Fluxes_{algo}"
  erddap_samos[algo].constraints = {
      "time>=": start_time,
      "time<=": end_time,
      "longitude>=": extent[0],
      "longitude<=": extent[1],
      "latitude>=": extent[2],
      "latitude<=": extent[3],
  }
  erddap_samos[algo].variables = [
      "platform_call_sign",
      "platform_name",
      "lhf",
      "lhf_qc",
      "shf",
      "shf_qc",
      "tau",
      "tau_qc"
  ]

### URLs for ERDDAP request can be retrieved

In [4]:
for algo in flux_algorithms:
    erddap_samos[algo].response = "csv"
    print(f'Download {algo} CSV: {erddap_samos[algo].get_download_url()}')

    erddap_samos[algo].response = "html"
    print(f'View {algo} download form on ERDDAP server: {erddap_samos[algo].get_download_url()}\n')

Download S88 CSV: http://marineflux-erddap.coaps.fsu.edu/erddap/tabledap/SAMOS_Fluxes_S88.csv?platform_call_sign,platform_name,lhf,lhf_qc,shf,shf_qc,tau,tau_qc&time>=1420070400.0&time<=1422748800.0&longitude>=280.0&longitude<=300.0&latitude>=30.0&latitude<=45.0
View S88 download form on ERDDAP server: http://marineflux-erddap.coaps.fsu.edu/erddap/tabledap/SAMOS_Fluxes_S88.html?platform_call_sign,platform_name,lhf,lhf_qc,shf,shf_qc,tau,tau_qc&time>=1420070400.0&time<=1422748800.0&longitude>=280.0&longitude<=300.0&latitude>=30.0&latitude<=45.0

Download B23 CSV: http://marineflux-erddap.coaps.fsu.edu/erddap/tabledap/SAMOS_Fluxes_B23.csv?platform_call_sign,platform_name,lhf,lhf_qc,shf,shf_qc,tau,tau_qc&time>=1420070400.0&time<=1422748800.0&longitude>=280.0&longitude<=300.0&latitude>=30.0&latitude<=45.0
View B23 download form on ERDDAP server: http://marineflux-erddap.coaps.fsu.edu/erddap/tabledap/SAMOS_Fluxes_B23.html?platform_call_sign,platform_name,lhf,lhf_qc,shf,shf_qc,tau,tau_qc&time>

### Request data as CF compliant netCDF4-python object

In [5]:
nc = dict()

for algo in flux_algorithms:
    nc[algo] = erddap_samos[algo].to_ncCF()

    print(nc[algo])

HTTPError: Error {
    code=404;
    message="Not Found: Your query produced no matching results. (nRows = 0)";
}


## Request data as pandas DataFrame

In [ ]:
df = dict()

for algo in flux_algorithms:
    df[algo] = erddap_samos[algo].to_pandas()

    print(df[algo])

## Request data as xarray Dataset

In [ ]:
ds = dict()

for algo in flux_algorithms:
    ds[algo] = erddap_samos[algo].to_xarray(requests_kwargs={"timeout": 600})

    print(ds[algo])

### Show which vessels are present in this subset

In [ ]:
print('\n'.join(map(str,list(set(zip(ds['C36']['platform_name'].data,ds['C36']['platform_call_sign'].data))))))

### Filter out data not flagged as "good"

In [9]:
vars = ['lhf', 'shf', 'tau']

for var in vars:
    
    for algo in flux_algorithms:

        if f'{var}_qc' in ds[algo]:
            good_indices = np.where((ds[algo][f'{var}_qc'].cf == 'good') == True)[0]
            mask = np.zeros(len(ds[algo][f'{var}'].data), dtype=bool)
            mask[good_indices] = True
            ds[algo][var].data[~mask] = np.nan
        

### Latent heat flux plotted as a function of time

In [ ]:
for algo in flux_algorithms:
    ds[algo]['lhf'].plot(x='time')

### Sensible and latent heat fluxes plotted on a map

In [11]:
def plot_shared_colorbar(dataset, vars, cmap='bwr', diverging=True, sort=False):
    height = 4
    figsize = (len(vars) * height, height)
    fig = plt.figure(figsize=figsize, layout='constrained')
    fig.suptitle(dataset.title)

    projection = ccrs.PlateCarree()

    ax_dict = fig.subplot_mosaic(
        [
            vars,
            ['cbar' for var in vars]
        ],
        height_ratios=[20, 1],
        per_subplot_kw={
            tuple(vars): {'projection': projection}
        }
    )

    vmax = -1
    for var in vars:
        ax_dict[var].set_extent(extent, crs=projection)
        ax_dict[var].stock_img()
        ax_dict[var].coastlines()
        ax_dict[var].set_title(f'{dataset[var].long_name} ({dataset[var].units})')

        vmax = max(vmax, ma.max(abs(ma.masked_invalid(ds[algo][var]))))

    if diverging:
        vmin = -vmax
    else:
        vmin = 0

    scatter = []
    for var in vars:
        # sorting the DataArray before plotting makes the higher values show up on top. this is useful for seeing where the high values are, but obscures the negative values.
        if sort:
            da = dataset[var].sortby(dataset[var])
        else:
            da = dataset[var]
        scatter.append(ax_dict[var].scatter(x=da['longitude'], y=da['latitude'], c=da, cmap=cmap, vmin=vmin, vmax=vmax))

    cbar = plt.colorbar(scatter[0], cax=ax_dict['cbar'], orientation='horizontal')

In [ ]:
for algo in flux_algorithms:
    plot_shared_colorbar(ds[algo], ['shf', 'lhf'], sort=True)

In [ ]:
for algo in flux_algorithms:
    plot_shared_colorbar(ds[algo], ['tau'], cmap='viridis', diverging=False, sort=True)

In [14]:
def plot_dataset_comparison(datasets, var, sort=False, cmap='bwr', diverging=True):
    keys = list(datasets.keys())
    height = 4
    figsize = (len(datasets) * height, height)
    fig = plt.figure(figsize=figsize, layout='constrained')
    fig.suptitle(f'{ds[keys[0]][var].long_name} ({ds[keys[0]][var].units})')

    projection = ccrs.PlateCarree()

    ax_dict = fig.subplot_mosaic(
        [
            [algo for algo in keys],
            ['cbar' for algo in keys]
        ],
        height_ratios=[20, 1],
        per_subplot_kw={
            tuple(algo for algo in keys): {'projection': projection}
        }
    )

    vmax = -1
    for algo in keys:
            
        ax_dict[algo].set_extent(extent, crs=projection)
        ax_dict[algo].stock_img()
        ax_dict[algo].coastlines()
        ax_dict[algo].set_title(f'{datasets[algo][var].long_name} ({datasets[algo][var].units})')

        vmax = max(vmax, ma.max(abs(ma.masked_invalid(datasets[algo][var]))))

        ax_dict[algo].set_title(datasets[algo].title)

    if diverging:
        vmin = -vmax
    else:
        vmin = 0

    scatter = []
    for algo in keys:
        # sorting the DataArray before plotting makes the higher values show up on top. this is useful for seeing where the high values are, but obscures the negative values.
        if sort:
            da = datasets[algo][var].sortby(datasets[algo][var])
        else:
            da = datasets[algo][var]
        scatter.append(ax_dict[algo].scatter(x=da['longitude'], y=da['latitude'], c=da, cmap=cmap, vmin=vmin, vmax=vmax))

    cbar = plt.colorbar(scatter[0], cax=ax_dict['cbar'], orientation='horizontal')

In [ ]:
plot_dataset_comparison(ds, 'lhf', sort=True)

In [ ]:
plot_dataset_comparison(ds, 'tau', cmap='viridis', diverging=False, sort=True)